### Year 1 Topology Analysis

In year 2, there are no new resource added to the system, there is increase in forecasted loads for all areas. The solar farm in Area 1 is operational. As solar is an intermittent resource, the generation from it is not constant. In this analysis, for various solar irradiances and forecasted load scenarios, area totals and corresponding load and generations are obtained by conducting load flow in PSSE. 

In [2]:
import pandas as pd

In [5]:
def area_select(area_name):
    '''
    function to select area data by area name
    '''
    list_area = []
    list_gens = [0,50,100]
    list_lsc = ['LLS','RLS','HLS']
    for gen in list_gens:
        for lsc in list_lsc:
            file_csv_in = 'savnw_sol_' + str(gen) +'_'+ lsc + '_tot.csv'
            data_tot = pd.read_csv(file_csv_in)
            data_tot['Scenario'] = 'Solar = ' + str(gen) + ' MW, ' + lsc.upper()
            data_area = data_tot[data_tot['Area'].str.strip() == area_name]
            list_area.append(data_area)
    df_area = pd.concat(list_area)
    df = df_area[['Scenario','Area', 'Number', 'From Generation', 'To Load', 'To(+)/From(-) Ties',
           'To Losses']].reset_index(drop=True)
    return df

##### Area Selection

Filtering out totals for each scenarios by area

In [6]:
area_select('FLAPCO')

,Scenario,Area,Number,From Generation,To Load,To(+)/From(-) Ties,To Losses
0,"Solar = 0 MW, LLS",FLAPCO,1,1323,1057,249,15
1,"Solar = 0 MW, RLS",FLAPCO,1,1383,1115,250,17
2,"Solar = 0 MW, HLS",FLAPCO,1,1431,1162,251,18
3,"Solar = 50 MW, LLS",FLAPCO,1,1322,1057,249,14
4,"Solar = 50 MW, RLS",FLAPCO,1,1382,1115,250,16
5,"Solar = 50 MW, HLS",FLAPCO,1,1430,1162,251,16
6,"Solar = 100 MW, LLS",FLAPCO,1,1321,1057,249,13
7,"Solar = 100 MW, RLS",FLAPCO,1,1381,1115,250,15
8,"Solar = 100 MW, HLS",FLAPCO,1,1429,1162,251,15


In [7]:
area_select('LIGHTCO')

,Scenario,Area,Number,From Generation,To Load,To(+)/From(-) Ties,To Losses
0,"Solar = 0 MW, LLS",LIGHTCO,2,1244,1322,-99,22
1,"Solar = 0 MW, RLS",LIGHTCO,2,1318,1394,-100,23
2,"Solar = 0 MW, HLS",LIGHTCO,2,1377,1452,-100,25
3,"Solar = 50 MW, LLS",LIGHTCO,2,1243,1322,-99,21
4,"Solar = 50 MW, RLS",LIGHTCO,2,1317,1394,-100,22
5,"Solar = 50 MW, HLS",LIGHTCO,2,1376,1452,-100,24
6,"Solar = 100 MW, LLS",LIGHTCO,2,1242,1322,-99,20
7,"Solar = 100 MW, RLS",LIGHTCO,2,1316,1394,-100,21
8,"Solar = 100 MW, HLS",LIGHTCO,2,1375,1452,-100,23


In [8]:
area_select('WORLD')

,Scenario,Area,Number,From Generation,To Load,To(+)/From(-) Ties,To Losses
0,"Solar = 0 MW, LLS",WORLD,5,294,440,-150,4
1,"Solar = 0 MW, RLS",WORLD,5,319,464,-150,4
2,"Solar = 0 MW, HLS",WORLD,5,338,484,-150,5
3,"Solar = 50 MW, LLS",WORLD,5,294,440,-150,4
4,"Solar = 50 MW, RLS",WORLD,5,319,464,-150,4
5,"Solar = 50 MW, HLS",WORLD,5,338,484,-150,5
6,"Solar = 100 MW, LLS",WORLD,5,294,440,-150,4
7,"Solar = 100 MW, RLS",WORLD,5,319,464,-150,4
8,"Solar = 100 MW, HLS",WORLD,5,338,484,-151,4


##### Total generation and Load

In [9]:
list_scenario = []
list_gen = []
list_load = []
list_gens = [0,50,100]
list_lsc = ['LLS','RLS','HLS']
for gen in list_gens:
    for lsc in list_lsc:
        file_csv_in = 'savnw_sol_' + str(gen) +'_'+ lsc + '_tot.csv'
        data_tot = pd.read_csv(file_csv_in)
        data_gen_load = pd.DataFrame()
        scenario = 'Solar = ' + str(gen) + ' MW, ' + lsc.upper()
        list_scenario.append(scenario)
        generation = data_tot['From Generation'].sum()
        list_gen.append(generation)
        load = data_tot['To Load'].sum()
        list_load.append(load)
gen_load_dict = {'Scenario':list_scenario, 
                 'Generation':list_gen,
                 'Load':list_load}
data_gen_load = pd.DataFrame(data=gen_load_dict)
data_gen_load

,Scenario,Generation,Load
0,"Solar = 0 MW, LLS",2861,2819
1,"Solar = 0 MW, RLS",3020,2973
2,"Solar = 0 MW, HLS",3146,3098
3,"Solar = 50 MW, LLS",2859,2819
4,"Solar = 50 MW, RLS",3018,2973
5,"Solar = 50 MW, HLS",3144,3098
6,"Solar = 100 MW, LLS",2857,2819
7,"Solar = 100 MW, RLS",3016,2973
8,"Solar = 100 MW, HLS",3142,3098


##### Breakdown of generation of each generators 

The generators in the hypothetical SAVNW system are 'NUC-A', 'NUC-B', 'SOLAR_PV', 'URBGEN', 'HYDRO_G', 'MINE_G', 'CATDOG_G'. The generators NUC-A, URBGEN and MINE_G are the swing buses of areas 1, 2, 5 respectively. The contribution of each of the generators to considered load scenarios are obtained.

In [10]:
def gen_contribution(name):
    '''
    Function to separate contribution of each generators to meet the demand in 
    various solar outputs and load scenarios
    '''
    list_gen = []
    list_gens = [0,50,100]
    list_lsc = ['LLS','RLS','HLS']
    for gen in list_gens:
        for lsc in list_lsc:
            file_csv_in = 'savnw_sol_' + str(gen) +'_'+ lsc + '_gen.csv'
            data_gen = pd.read_csv(file_csv_in, header=1, skiprows=0)
            data_gen['Scenario'] = 'Solar = ' + str(gen) + ' MW, ' + lsc.upper()
            data_new = data_gen[(data_gen['Name'].str.strip()) == name]
            list_gen.append(data_new)
           
    df_area = pd.concat(list_gen)
    df = df_area[['Scenario','Number', 'Name', 'Area', 'Pgen', 'Pmax', 'Reserve']].reset_index(drop=True)
    return df

In [11]:
gen_contribution('NUC-A')

,Scenario,Number,Name,Area,Pgen,Pmax,Reserve
0,"Solar = 0 MW, LLS",101,NUC-A,1,623.03,810.0,186.97
1,"Solar = 0 MW, RLS",101,NUC-A,1,683.13,810.0,126.87
2,"Solar = 0 MW, HLS",101,NUC-A,1,731.88,810.0,78.12
3,"Solar = 50 MW, LLS",101,NUC-A,1,572.28,810.0,237.72
4,"Solar = 50 MW, RLS",101,NUC-A,1,632.06,810.0,177.94
5,"Solar = 50 MW, HLS",101,NUC-A,1,680.81,810.0,129.19
6,"Solar = 100 MW, LLS",101,NUC-A,1,521.38,810.0,288.62
7,"Solar = 100 MW, RLS",101,NUC-A,1,581.06,810.0,228.94
8,"Solar = 100 MW, HLS",101,NUC-A,1,629.80,810.0,180.20


In [12]:
gen_contribution('NUC-B')

,Scenario,Number,Name,Area,Pgen,Pmax,Reserve
0,"Solar = 0 MW, LLS",102,NUC-B,1,700.0,810.0,110.0
1,"Solar = 0 MW, RLS",102,NUC-B,1,700.0,810.0,110.0
2,"Solar = 0 MW, HLS",102,NUC-B,1,700.0,810.0,110.0
3,"Solar = 50 MW, LLS",102,NUC-B,1,700.0,810.0,110.0
4,"Solar = 50 MW, RLS",102,NUC-B,1,700.0,810.0,110.0
5,"Solar = 50 MW, HLS",102,NUC-B,1,700.0,810.0,110.0
6,"Solar = 100 MW, LLS",102,NUC-B,1,700.0,810.0,110.0
7,"Solar = 100 MW, RLS",102,NUC-B,1,700.0,810.0,110.0
8,"Solar = 100 MW, HLS",102,NUC-B,1,700.0,810.0,110.0


The reserve capacity obtained with solar power plant is not dependable capacity, and hence wont contribute to firm capacity of the system. 

In [13]:
gen_contribution('SOLAR_PV')

,Scenario,Number,Name,Area,Pgen,Pmax,Reserve
0,"Solar = 0 MW, LLS",103,SOLAR_PV,1,0.0,117.0,117.0
1,"Solar = 0 MW, RLS",103,SOLAR_PV,1,0.0,117.0,117.0
2,"Solar = 0 MW, HLS",103,SOLAR_PV,1,0.0,117.0,117.0
3,"Solar = 50 MW, LLS",103,SOLAR_PV,1,50.0,117.0,67.0
4,"Solar = 50 MW, RLS",103,SOLAR_PV,1,50.0,117.0,67.0
5,"Solar = 50 MW, HLS",103,SOLAR_PV,1,50.0,117.0,67.0
6,"Solar = 100 MW, LLS",103,SOLAR_PV,1,100.0,117.0,17.0
7,"Solar = 100 MW, RLS",103,SOLAR_PV,1,100.0,117.0,17.0
8,"Solar = 100 MW, HLS",103,SOLAR_PV,1,100.0,117.0,17.0


In [14]:
gen_contribution('URBGEN')

,Scenario,Number,Name,Area,Pgen,Pmax,Reserve
0,"Solar = 0 MW, LLS",206,URBGEN,2,744.90,900.0,155.10
1,"Solar = 0 MW, RLS",206,URBGEN,2,818.29,900.0,81.71
2,"Solar = 0 MW, HLS",206,URBGEN,2,877.49,900.0,22.51
3,"Solar = 50 MW, LLS",206,URBGEN,2,743.48,900.0,156.52
4,"Solar = 50 MW, RLS",206,URBGEN,2,817.17,900.0,82.83
5,"Solar = 50 MW, HLS",206,URBGEN,2,876.37,900.0,23.63
6,"Solar = 100 MW, LLS",206,URBGEN,2,742.46,900.0,157.54
7,"Solar = 100 MW, RLS",206,URBGEN,2,816.10,900.0,83.90
8,"Solar = 100 MW, HLS",206,URBGEN,2,875.32,900.0,24.68


In [15]:
gen_contribution('HYDRO_G')

,Scenario,Number,Name,Area,Pgen,Pmax,Reserve
0,"Solar = 0 MW, LLS",211,HYDRO_G,2,500.0,616.25,116.25
1,"Solar = 0 MW, RLS",211,HYDRO_G,2,500.0,616.25,116.25
2,"Solar = 0 MW, HLS",211,HYDRO_G,2,500.0,616.25,116.25
3,"Solar = 50 MW, LLS",211,HYDRO_G,2,500.0,616.25,116.25
4,"Solar = 50 MW, RLS",211,HYDRO_G,2,500.0,616.25,116.25
5,"Solar = 50 MW, HLS",211,HYDRO_G,2,500.0,616.25,116.25
6,"Solar = 100 MW, LLS",211,HYDRO_G,2,500.0,616.25,116.25
7,"Solar = 100 MW, RLS",211,HYDRO_G,2,500.0,616.25,116.25
8,"Solar = 100 MW, HLS",211,HYDRO_G,2,500.0,616.25,116.25


In [16]:
gen_contribution('MINE_G')

,Scenario,Number,Name,Area,Pgen,Pmax,Reserve
0,"Solar = 0 MW, LLS",3011,MINE_G,5,204.82,900.0,695.18
1,"Solar = 0 MW, RLS",3011,MINE_G,5,229.65,900.0,670.35
2,"Solar = 0 MW, HLS",3011,MINE_G,5,248.59,900.0,651.41
3,"Solar = 50 MW, LLS",3011,MINE_G,5,204.84,900.0,695.16
4,"Solar = 50 MW, RLS",3011,MINE_G,5,229.50,900.0,670.50
5,"Solar = 50 MW, HLS",3011,MINE_G,5,248.32,900.0,651.68
6,"Solar = 100 MW, LLS",3011,MINE_G,5,204.68,900.0,695.32
7,"Solar = 100 MW, RLS",3011,MINE_G,5,229.37,900.0,670.63
8,"Solar = 100 MW, HLS",3011,MINE_G,5,248.08,900.0,651.92


In [17]:
gen_contribution('CATDOG_G')

,Scenario,Number,Name,Area,Pgen,Pmax,Reserve
0,"Solar = 0 MW, LLS",3018,CATDOG_G,5,90.0,117.0,27.0
1,"Solar = 0 MW, RLS",3018,CATDOG_G,5,90.0,117.0,27.0
2,"Solar = 0 MW, HLS",3018,CATDOG_G,5,90.0,117.0,27.0
3,"Solar = 50 MW, LLS",3018,CATDOG_G,5,90.0,117.0,27.0
4,"Solar = 50 MW, RLS",3018,CATDOG_G,5,90.0,117.0,27.0
5,"Solar = 50 MW, HLS",3018,CATDOG_G,5,90.0,117.0,27.0
6,"Solar = 100 MW, LLS",3018,CATDOG_G,5,90.0,117.0,27.0
7,"Solar = 100 MW, RLS",3018,CATDOG_G,5,90.0,117.0,27.0
8,"Solar = 100 MW, HLS",3018,CATDOG_G,5,90.0,117.0,27.0


##### Firm and Non-firm Installed Capacity

In [18]:
firm_capacity = gen_contribution('NUC-A').loc[0,'Pmax'] + gen_contribution('NUC-B').loc[0,'Pmax'] +\
          gen_contribution('HYDRO_G').loc[0,'Pmax'] + gen_contribution('URBGEN').loc[0,'Pmax'] +\
          gen_contribution('MINE_G').loc[0,'Pmax'] + gen_contribution('CATDOG_G').loc[0,'Pmax'] 
firm_capacity 

4153.25

In [19]:
non_firm_capacity = gen_contribution('SOLAR_PV').loc[0,'Pmax']
non_firm_capacity

117.0

##### Load distribution of the three studied Load scenarios

In [20]:
def data_lsc(data_excel, year,strg):
    data = pd.read_excel(data_excel)
    pload = 'Pload-Y'+str(year)+strg
    qload = 'Qload-Y'+str(year)+strg
    data_fil = data[['Load','ID',pload,qload]]
    return data_fil

##### Low Load Scenario

In [22]:
data_lls = data_lsc('lls.xlsx',2,'L')
data_lls

,Load,ID,Pload-Y2L,Qload-Y2L
0,153,1,176.27155,88.17900
1,154,1,528.90110,396.80550
2,154,2,352.54310,308.62650
3,203,1,264.36410,132.26850
4,205,1,1057.71575,617.25300
5,3005,1,88.09255,44.08950
6,3007,1,176.27155,66.13425
7,3008,1,176.27155,66.13425


In [25]:
total_load = data_lls['Pload-Y2L'].sum().round(2)
total_load

2820.43

##### Reference Load Scenario

In [27]:
data_rls = data_lsc('rls.xlsx',2,'R')
data_rls

,Load,ID,Pload-Y2R,Qload-Y2R
0,153,1,185.9568,93.024
1,154,1,557.9616,418.608
2,154,2,371.9136,325.584
3,203,1,278.8896,139.536
4,205,1,1115.8320,651.168
5,3005,1,92.9328,46.512
6,3007,1,185.9568,69.768
7,3008,1,185.9568,69.768


In [28]:
total_load = data_rls['Pload-Y2R'].sum()
total_load

2975.4

##### High Load Scenario

In [29]:
data_hls = data_lsc('hls.xlsx',2,'H')
data_hls

,Load,ID,Pload-Y2H,Qload-Y2H
0,153,1,193.705,96.900
1,154,1,581.210,436.050
2,154,2,387.410,339.150
3,203,1,290.510,145.350
4,205,1,1162.325,678.300
5,3005,1,96.805,48.450
6,3007,1,193.705,72.675
7,3008,1,193.705,72.675


In [30]:
total_load = data_hls['Pload-Y2H'].sum()
total_load

3099.375

##### Conclusion

Year 1 Toplogy Analysis was carried out for 3 solar outputs and 3 load scenarios - High Load Scenario, Low Load Scenario and Reference Load Scenario and determined 
1. generation and load totals for each scenarios by area
2. contribution of each of the generators to considered load scenarios
3. firm and non-firm capacity contributions of installed resources
4. total load for the three scenarios